In [1]:
global Path
if sc.master[0:5]=="local":
    Path="file:///home/vmauser/pythonwork/PythonProject/"
else:
    Path="file:/home/vmauser/"

### Data preprocessing

#### Importing data

In [2]:
row_df=sqlContext.read.format("csv").option("header", "true").option(
    "delimiter", "\t").load(Path+"data/train.tsv")

print(row_df.count())

7395


In [3]:
row_df.printSchema()

root
 |-- url: string (nullable = true)
 |-- urlid: string (nullable = true)
 |-- boilerplate: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: string (nullable = true)
 |-- avglinksize: string (nullable = true)
 |-- commonlinkratio_1: string (nullable = true)
 |-- commonlinkratio_2: string (nullable = true)
 |-- commonlinkratio_3: string (nullable = true)
 |-- commonlinkratio_4: string (nullable = true)
 |-- compression_ratio: string (nullable = true)
 |-- embed_ratio: string (nullable = true)
 |-- framebased: string (nullable = true)
 |-- frameTagRatio: string (nullable = true)
 |-- hasDomainLink: string (nullable = true)
 |-- html_ratio: string (nullable = true)
 |-- image_ratio: string (nullable = true)
 |-- is_news: string (nullable = true)
 |-- lengthyLinkDomain: string (nullable = true)
 |-- linkwordscore: string (nullable = true)
 |-- news_front_page: string (nullable = true)
 |-- non_markup_alphanum_characters: string (nulla

In [4]:
row_df.select("url", "alchemy_category", "alchemy_category_score", "is_news", "label").show(10)

+--------------------+------------------+----------------------+-------+-----+
|                 url|  alchemy_category|alchemy_category_score|is_news|label|
+--------------------+------------------+----------------------+-------+-----+
|http://www.bloomb...|          business|              0.789131|      1|    0|
|http://www.popsci...|        recreation|              0.574147|      1|    1|
|http://www.menshe...|            health|              0.996526|      1|    1|
|http://www.dumbli...|            health|              0.801248|      1|    1|
|http://bleacherre...|            sports|              0.719157|      1|    0|
|http://www.conven...|                 ?|                     ?|      ?|    0|
|http://gofashionl...|arts_entertainment|               0.22111|      1|    1|
|http://www.inside...|                 ?|                     ?|      ?|    0|
|http://www.valetm...|                 ?|                     ?|      1|    1|
|http://www.howswe...|                 ?|           

#### Type transformation

In [5]:
from pyspark.sql.functions import udf

def replace_question(x):
    return ("0" if x=="?" else x)

replace_question=udf(replace_question)

In [6]:
from pyspark.sql.functions import col
import pyspark.sql.types

df=row_df.select(["url", "alchemy_category"]+
                 [replace_question(col(column)).cast("double").alias(column) 
                  for column in row_df.columns[4:]])

print df.printSchema()

root
 |-- url: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: double (nullable = true)
 |-- avglinksize: double (nullable = true)
 |-- commonlinkratio_1: double (nullable = true)
 |-- commonlinkratio_2: double (nullable = true)
 |-- commonlinkratio_3: double (nullable = true)
 |-- commonlinkratio_4: double (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- embed_ratio: double (nullable = true)
 |-- framebased: double (nullable = true)
 |-- frameTagRatio: double (nullable = true)
 |-- hasDomainLink: double (nullable = true)
 |-- html_ratio: double (nullable = true)
 |-- image_ratio: double (nullable = true)
 |-- is_news: double (nullable = true)
 |-- lengthyLinkDomain: double (nullable = true)
 |-- linkwordscore: double (nullable = true)
 |-- news_front_page: double (nullable = true)
 |-- non_markup_alphanum_characters: double (nullable = true)
 |-- numberOfLinks: double (nullable = true)
 |-- numwords_in_url: d

In [7]:
df.select("url", "alchemy_category", "alchemy_category_score", "is_news", "label").show(10)

+--------------------+------------------+----------------------+-------+-----+
|                 url|  alchemy_category|alchemy_category_score|is_news|label|
+--------------------+------------------+----------------------+-------+-----+
|http://www.bloomb...|          business|              0.789131|    1.0|  0.0|
|http://www.popsci...|        recreation|              0.574147|    1.0|  1.0|
|http://www.menshe...|            health|              0.996526|    1.0|  1.0|
|http://www.dumbli...|            health|              0.801248|    1.0|  1.0|
|http://bleacherre...|            sports|              0.719157|    1.0|  0.0|
|http://www.conven...|                 ?|                   0.0|    0.0|  0.0|
|http://gofashionl...|arts_entertainment|               0.22111|    1.0|  1.0|
|http://www.inside...|                 ?|                   0.0|    0.0|  0.0|
|http://www.valetm...|                 ?|                   0.0|    1.0|  1.0|
|http://www.howswe...|                 ?|           

#### Train/test splitting

In [8]:
train_df, test_df=df.randomSplit([0.7, 0.3])
train_df.cache()
test_df.cache()

DataFrame[url: string, alchemy_category: string, alchemy_category_score: double, avglinksize: double, commonlinkratio_1: double, commonlinkratio_2: double, commonlinkratio_3: double, commonlinkratio_4: double, compression_ratio: double, embed_ratio: double, framebased: double, frameTagRatio: double, hasDomainLink: double, html_ratio: double, image_ratio: double, is_news: double, lengthyLinkDomain: double, linkwordscore: double, news_front_page: double, non_markup_alphanum_characters: double, numberOfLinks: double, numwords_in_url: double, parametrizedLinkRatio: double, spelling_errors_ratio: double, label: double]

### Feature engineering

#### StringIndexer

In [9]:
from pyspark.ml.feature import StringIndexer

categoryIndexer=StringIndexer(inputCol="alchemy_category", outputCol="alchemy_category_Index")
caregoryTransfomer=categoryIndexer.fit(df)

for i in range(0, len(caregoryTransfomer.labels)):
    print(str(i)+":"+caregoryTransfomer.labels[i])

0:?
1:recreation
2:arts_entertainment
3:business
4:health
5:sports
6:culture_politics
7:computer_internet
8:science_technology
9:gaming
10:religion
11:law_crime
12:unknown
13:weather


In [10]:
df1=caregoryTransfomer.transform(train_df)
print(df1.columns)

['url', 'alchemy_category', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio', 'label', 'alchemy_category_Index']


In [11]:
df1.select("alchemy_category", "alchemy_category_Index").show(10)

+------------------+----------------------+
|  alchemy_category|alchemy_category_Index|
+------------------+----------------------+
|                 ?|                   0.0|
|arts_entertainment|                   2.0|
|        recreation|                   1.0|
|                 ?|                   0.0|
|                 ?|                   0.0|
|                 ?|                   0.0|
|          business|                   3.0|
|arts_entertainment|                   2.0|
|                 ?|                   0.0|
|arts_entertainment|                   2.0|
+------------------+----------------------+
only showing top 10 rows



#### OneHotEncoder

In [12]:
from pyspark.ml.feature import OneHotEncoder

encoder=OneHotEncoder(dropLast=False, inputCol="alchemy_category_Index",
                      outputCol="alchemy_category_IndexVec")

df2=encoder.transform(df1)
print(df2.columns)

['url', 'alchemy_category', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio', 'label', 'alchemy_category_Index', 'alchemy_category_IndexVec']


In [13]:
df2.select("alchemy_category", "alchemy_category_Index", "alchemy_category_IndexVec").show(10)

+------------------+----------------------+-------------------------+
|  alchemy_category|alchemy_category_Index|alchemy_category_IndexVec|
+------------------+----------------------+-------------------------+
|                 ?|                   0.0|           (14,[0],[1.0])|
|arts_entertainment|                   2.0|           (14,[2],[1.0])|
|        recreation|                   1.0|           (14,[1],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|          business|                   3.0|           (14,[3],[1.0])|
|arts_entertainment|                   2.0|           (14,[2],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|arts_entertainment|                   2.0|           (14,[2],[1.0])|
+------------------+----------------------+-------------------------+
only showing top 10 

#### VectorAssembler

In [14]:
from pyspark.ml.feature import VectorAssembler

assemblerInputs=["alchemy_category_IndexVec"]+df.columns[2:-1]
print(assemblerInputs)

['alchemy_category_IndexVec', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio']


In [15]:
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")
df3=assembler.transform(df2)
print(df3.columns)

['url', 'alchemy_category', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio', 'label', 'alchemy_category_Index', 'alchemy_category_IndexVec', 'features']


In [16]:
df3.select("features").show(5)
df3.select("features").take(1)

+--------------------+
|            features|
+--------------------+
|(36,[0,15,16,17,1...|
|(36,[2,14,15,16,1...|
|(36,[1,14,15,16,1...|
|(36,[0,15,20,23,2...|
|(36,[0,15,16,20,2...|
+--------------------+
only showing top 5 rows



[Row(features=SparseVector(36, {0: 1.0, 15: 2.1446, 16: 0.7969, 17: 0.3945, 18: 0.332, 19: 0.3203, 20: 0.5022, 23: 0.028, 25: 0.1898, 26: 0.2354, 27: 1.0, 28: 1.0, 29: 17.0, 31: 10588.0, 32: 256.0, 33: 5.0, 34: 0.3828, 35: 0.1368}))]

### Modeling

In [17]:
from pyspark.ml.classification import DecisionTreeClassifier

dt=DecisionTreeClassifier(labelCol="label", featuresCol="features",
                          impurity="gini", maxDepth=10, maxBins=14)

dt_model=dt.fit(df3)
print(dt_model)

#df4=dt_model.transform(df3)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_542ab18a40af) of depth 10 with 583 nodes


### Using ML pipeline

#### Decision tree

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

categoryIndexer=StringIndexer(inputCol="alchemy_category", outputCol="alchemy_category_Index")

encoder=OneHotEncoder(dropLast=False, inputCol="alchemy_category_Index", 
                      outputCol="alchemy_category_IndexVec")

assemblerInputs=["alchemy_category_IndexVec"]+df.columns[2:-1]
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")

dt=DecisionTreeClassifier(labelCol="label", featuresCol="features", 
                          impurity="gini", maxDepth=10, maxBins=14)

pipeline=Pipeline(stages=[categoryIndexer, encoder, assembler, dt])
pipeline.getStages()

[StringIndexer_27bfadd9eeed,
 OneHotEncoder_28f094c74628,
 VectorAssembler_288fe2731d0c,
 DecisionTreeClassifier_c6fe4b5e1f3b]

In [19]:
pipelineModel=pipeline.fit(train_df)

In [20]:
pipelineModel.stages[3].toDebugString

u'DecisionTreeClassificationModel (uid=DecisionTreeClassifier_c6fe4b5e1f3b) of depth 10 with 583 nodes\n  If (feature 31 <= 2321.5)\n   If (feature 31 <= 1307.5)\n    If (feature 4 in {1.0})\n     If (feature 23 <= 0.1183033225)\n      If (feature 29 <= 65.5)\n       If (feature 35 <= 0.0547604445)\n        Predict: 0.0\n       Else (feature 35 > 0.0547604445)\n        If (feature 33 <= 9.5)\n         Predict: 1.0\n        Else (feature 33 > 9.5)\n         If (feature 16 <= 0.31650122950000004)\n          Predict: 1.0\n         Else (feature 16 > 0.31650122950000004)\n          Predict: 0.0\n      Else (feature 29 > 65.5)\n       If (feature 14 <= 0.4619955)\n        Predict: 0.0\n       Else (feature 14 > 0.4619955)\n        If (feature 19 <= 0.022753158)\n         Predict: 0.0\n        Else (feature 19 > 0.022753158)\n         Predict: 1.0\n     Else (feature 23 > 0.1183033225)\n      If (feature 35 <= 0.1322986505)\n       Predict: 0.0\n      Else (feature 35 > 0.1322986505)\n      

In [21]:
predicted=pipelineModel.transform(test_df)
print(predicted.columns)

['url', 'alchemy_category', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio', 'label', 'alchemy_category_Index', 'alchemy_category_IndexVec', 'features', 'rawPrediction', 'probability', 'prediction']


In [22]:
predicted.select("features", "rawPrediction", "probability", "label", "prediction").show(10)
predicted.select("probability", "prediction").take(10)

+--------------------+-------------+--------------------+-----+----------+
|            features|rawPrediction|         probability|label|prediction|
+--------------------+-------------+--------------------+-----+----------+
|(36,[0,15,16,17,1...|  [17.0,32.0]|[0.34693877551020...|  1.0|       1.0|
|(36,[3,14,15,16,1...|   [1.0,13.0]|[0.07142857142857...|  1.0|       1.0|
|(36,[3,14,15,16,1...|    [2.0,0.0]|           [1.0,0.0]|  1.0|       0.0|
|(36,[1,14,15,16,1...|   [13.0,4.0]|[0.76470588235294...|  0.0|       0.0|
|(36,[0,15,16,17,2...| [274.0,94.0]|[0.74456521739130...|  0.0|       0.0|
|(36,[0,15,16,17,1...|   [1.0,23.0]|[0.04166666666666...|  0.0|       1.0|
|(36,[0,15,16,17,1...|   [1.0,23.0]|[0.04166666666666...|  1.0|       1.0|
|(36,[0,15,16,17,1...|  [70.0,89.0]|[0.44025157232704...|  0.0|       1.0|
|(36,[2,14,15,16,1...|    [1.0,4.0]|           [0.2,0.8]|  0.0|       1.0|
|(36,[4,14,15,16,2...|    [2.0,0.0]|           [1.0,0.0]|  1.0|       0.0|
+--------------------+---

[Row(probability=DenseVector([0.3469, 0.6531]), prediction=1.0),
 Row(probability=DenseVector([0.0714, 0.9286]), prediction=1.0),
 Row(probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(probability=DenseVector([0.7647, 0.2353]), prediction=0.0),
 Row(probability=DenseVector([0.7446, 0.2554]), prediction=0.0),
 Row(probability=DenseVector([0.0417, 0.9583]), prediction=1.0),
 Row(probability=DenseVector([0.0417, 0.9583]), prediction=1.0),
 Row(probability=DenseVector([0.4403, 0.5597]), prediction=1.0),
 Row(probability=DenseVector([0.2, 0.8]), prediction=1.0),
 Row(probability=DenseVector([1.0, 0.0]), prediction=0.0)]

In [23]:
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", 
                                        labelCol="label", metricName="areaUnderROC")

auc=evaluator.evaluate(predicted)
auc

0.570610257508138

#### Decision tree with TrainVaildation

In [24]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

categoryIndexer=StringIndexer(inputCol="alchemy_category", outputCol="alchemy_category_Index")

encoder=OneHotEncoder(dropLast=False, inputCol="alchemy_category_Index", 
                      outputCol="alchemy_category_IndexVec")

assemblerInputs=["alchemy_category_IndexVec"]+df.columns[2:-1]
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")

dt=DecisionTreeClassifier(labelCol="label", featuresCol="features", 
                          impurity="gini", maxDepth=10, maxBins=14)

paramGrid=ParamGridBuilder().addGrid(dt.impurity, ["gini", "entropy"])\
.addGrid(dt.maxDepth, [5, 10, 15])\
.addGrid(dt.maxBins, [10, 15, 20])\
.build()

evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", 
                                        metricName="areaUnderROC")

tvs=TrainValidationSplit(estimator=dt, evaluator=evaluator, 
                         estimatorParamMaps=paramGrid, trainRatio=0.8)

tvs_pipeline=Pipeline(stages=[categoryIndexer, encoder, assembler, tvs])

tvs_pipelineModel=tvs_pipeline.fit(train_df)

bestModel=tvs_pipelineModel.stages[3].bestModel

bestModel.toDebugString[:200]

u'DecisionTreeClassificationModel (uid=DecisionTreeClassifier_c6fe4b5e1f3b) of depth 15 with 1603 nodes\n  If (feature 31 <= 2681.5)\n   If (feature 31 <= 1206.0)\n    If (feature 4 in {1.0})\n     If (feat'

In [25]:
predictions=tvs_pipelineModel.transform(test_df)

auc=evaluator.evaluate(predictions)
auc

0.6483457668428766

#### Decision tree with CrossVaildation

In [26]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

categoryIndexer=StringIndexer(inputCol="alchemy_category", outputCol="alchemy_category_Index")

encoder=OneHotEncoder(dropLast=False, inputCol="alchemy_category_Index", 
                      outputCol="alchemy_category_IndexVec")

assemblerInputs=["alchemy_category_IndexVec"]+df.columns[2:-1]
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")

dt=DecisionTreeClassifier(labelCol="label", featuresCol="features", 
                          impurity="gini", maxDepth=10, maxBins=14)

paramGrid=ParamGridBuilder().addGrid(dt.impurity, ["gini", "entropy"])\
.addGrid(dt.maxDepth, [5, 10, 15])\
.addGrid(dt.maxBins, [10, 15, 20])\
.build()

evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", 
                                        metricName="areaUnderROC")

cv=CrossValidator(estimator=dt, evaluator=evaluator,
                   estimatorParamMaps=paramGrid, numFolds=3)

cv_pipeline=Pipeline(stages=[categoryIndexer, encoder, assembler, cv])

cv_pipelineModel=cv_pipeline.fit(train_df)

bestModel=cv_pipelineModel.stages[3].bestModel
bestModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_c6fe4b5e1f3b) of depth 15 with 1495 nodes

In [27]:
predictions=cv_pipelineModel.transform(test_df)
auc=evaluator.evaluate(predictions)
auc

0.646693062878034

#### RandomForest

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

categoryIndexer=StringIndexer(inputCol="alchemy_category", outputCol="alchemy_category_Index")

encoder=OneHotEncoder(dropLast=False, inputCol="alchemy_category_Index",
                      outputCol="alchemy_category_IndexVec")

assemblerInputs=["alchemy_category_IndexVec"]+df.columns[2:-1]
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")

rf=RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

rfpipeline=Pipeline(stages=[categoryIndexer, encoder, assembler, rf])
rfpipeline.getStages()

[StringIndexer_4546249a9c1d,
 OneHotEncoder_6a404a8d61b4,
 VectorAssembler_c77ada5aff86,
 RandomForestClassifier_da63ae32342a]

In [10]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

rfpipelineModel=rfpipeline.fit(train_df)
rfpredicted=rfpipelineModel.transform(test_df)

evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", 
                                        metricName="areaUnderROC")

auc=evaluator.evaluate(rfpredicted)
auc

0.7384597211815173

#### Random forest with TrainValidation

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator

categoryIndexer=StringIndexer(inputCol="alchemy_category", outputCol="alchemy_category_Index")

encoder=OneHotEncoder(dropLast=False, inputCol="alchemy_category_Index",
                      outputCol="alchemy_category_IndexVec")

assemblerInputs=["alchemy_category_IndexVec"]+df.columns[2:-1]
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")

rf=RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

paramGrid=ParamGridBuilder().addGrid(rf.impurity, ["gini", "entropy"])\
.addGrid(rf.maxDepth, [5, 10, 15])\
.addGrid(rf.maxBins, [10, 15, 20])\
.addGrid(rf.numTrees, [10, 20, 30])\
.build()

evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", 
                                        metricName="areaUnderROC")

rftvs=TrainValidationSplit(estimator=rf, evaluator=evaluator, estimatorParamMaps=paramGrid,
                           trainRatio=0.8)

rftvs_pipeline=Pipeline(stages=[categoryIndexer, encoder, assembler, rftvs])

rftvs_pipelineModel=rftvs_pipeline.fit(train_df)

rftvspredictions=rftvs_pipelineModel.transform(test_df)

auc=evaluator.evaluate(rftvspredictions)
auc

0.7757623917999431

#### Random forest with CrossValidation

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

categoryIndexer=StringIndexer(inputCol="alchemy_category", outputCol="alchemy_category_Index")

encoder=OneHotEncoder(dropLast=False, inputCol="alchemy_category_Index",
                      outputCol="alchemy_category_IndexVec")

assemblerInputs=["alchemy_category_IndexVec"]+df.columns[2:-1]
assembler=VectorAssembler(inputCols=assemblerInputs, outputCol="features")

rf=RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

paramGrid=ParamGridBuilder().addGrid(rf.impurity, ["gini", "entropy"])\
.addGrid(rf.maxDepth, [5, 10, 15])\
.addGrid(rf.maxBins, [10, 15, 20])\
.addGrid(rf.numTrees, [10, 20, 30])\
.build()

evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", 
                                        metricName="areaUnderROC")

rfcv=CrossValidator(estimator=rf, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)

rfcv_pipeline=Pipeline(stages=[categoryIndexer, encoder, assembler, rfcv])

rfcv_pipelineModel=rfcv_pipeline.fit(train_df)

rfcvpredictions=rfcv_pipelineModel.transform(test_df)

auc=evaluator.evaluate(rfcvpredictions)
auc

0.764369667552788

In [13]:
DescDict={
    0: "Ephemeral",
    1: "Evergreen"
}

for data in rfcvpredictions.select("url", "prediction").take(5):
    print("web url: " + str(data[0]) + "\n" + \
         "              ==> prediction:" + str(data[1]) + \
         " description: " + DescDict[data[1]] + "\n")

web url: http://17andbaking.com/2010/04/02/blackberry-jam-almond-bars/
              ==> prediction:1.0 description: Evergreen

web url: http://1greengeneration.elementsintime.com/?p=365
              ==> prediction:1.0 description: Evergreen

web url: http://1x.com/photo/39383/portfolio/39932
              ==> prediction:0.0 description: Ephemeral

web url: http://24-7humor.com/what-the-hell-windows-95/
              ==> prediction:0.0 description: Ephemeral

web url: http://30aeats.com/recipes/sun-dried-tomato-kalamata-olive-and-basil-pesto-focaccia/
              ==> prediction:1.0 description: Evergreen

